In [8]:
from xai import *
import numpy as np
np.set_printoptions(precision=8, suppress=True)
import matplotlib.pyplot as plt
import random
import numpy as np
import torch
import shap
import pandas as pd
import plotly.express as px
from dateutil import parser as dateutil
from tqdm import tqdm

In [21]:
csv = pd.read_csv("new-york-exhange/prices.csv").sort_values("date")
csv

,date,symbol,open,close,low,high,volume
646,2010-01-04,SYMC,18.040001,18.400000,18.010000,18.530001,8322300.0
472,2010-01-04,IDXX,53.700001,54.080002,53.430000,54.349998,325200.0
473,2010-01-04,IFF,41.509998,42.009998,41.500000,42.020000,286000.0
474,2010-01-04,ILMN,31.120001,30.549999,30.420000,31.230000,1793700.0
475,2010-01-04,INTC,20.790001,20.879999,20.730000,21.030001,47800900.0
...,...,...,...,...,...,...,...
850932,2016-12-30,EXPE,113.839996,113.279999,112.720001,114.239998,2082700.0
851260,2016-12-30,ZION,43.070000,43.040001,42.689999,43.310001,1938100.0
250,2016-12-30 00:00:00,WLTW,122.589996,122.279999,121.389999,123.559998,466400.0
851262,2016-12-30 00:00:00,AIV,44.730000,45.450001,44.410000,45.590000,1380900.0


In [73]:
years: dict[int,dict[int,dict[int,dict[str,list[float]]]]] = {}

max_volume = csv["volume"].max()

with tqdm(total=len(csv)) as bar:
    for i,(date,symbol,open,close,low,high,volume) in csv.iterrows():
        date = dateutil.parse(date)

        months: dict = years.setdefault(date.year, {})
        days: dict = months.setdefault(date.month, {})
        symbols: dict = days.setdefault(date.day, {})
        symbols.setdefault(symbol, [close-open, high-low, volume/max_volume])

        bar.update()

100%|██████████| 851264/851264 [01:27<00:00, 9714.17it/s] 


In [118]:
years[2015][3][5].__len__()

494

In [123]:
listed = []

symbol_cnt = csv["symbol"].unique().size

with tqdm(total=len(csv)) as bar:
    for month in years.values():
        for days in month.values():
            for symbols in days.values():
                flattened = []
                for values in symbols.values():
                    flattened.extend(values)

                listed.extend(flattened)

        bar.update()

  0%|          | 0/851264 [00:00<?, ?it/s]

  0%|          | 7/851264 [00:00<5:06:32, 46.28it/s]


In [124]:
len(listed)

2553792

In [136]:
date_idx = np.arange(csv["date"].unique().size)
symbol_idx = np.arange(csv["symbol"].unique().size)

dataset = np.zeros((len(date_idx), len(symbol_idx), 3), dtype=np.float32)

for i,(date,symbol,open,close,low,high,volume) in csv.iterrows():
    print(date)

646
472
473
474
475
476
477
478
479
471
480
482
483
484
485
486
487
488
489
481
490
470
468
450
451
452
453
454
455
456
457
469
458
460
461
462
463
464
465
466
467
459
491
492
493
517
518
520
521
522
523
524
525
516
526
528
529
530
531
532
533
534
535
527
515
514
513
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
449
448
447
446
383
384
385
386
387
388
389
390
382
391
393
394
395
396
397
398
399
400
392
381
380
379
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
401
536
402
404
428
429
430
431
432
433
434
435
427
436
438
439
440
441
442
443
444
445
437
426
425
424
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
403
537
538
539
654
655
656
657
658
659
660
661
653
662
664
665
666
667
668
669
670
671
663
652
651
650
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
647
648
649
672
629
673
675
699
700
701
702
703
704
705
706
698
707
709
710
711
712
713
714
715
716
708
697
696
695
676
677
678


KeyboardInterrupt: 

In [117]:
date_idx

array([   0,    1,    2, ..., 3521, 3522, 3523])

In [43]:
cooperations_cnt = df["symbol"].unique().size
price_change = df["open"] - df["close"]

In [41]:
price_change

0        -2.409996
1         5.259995
2         1.430000
3        -1.140000
4         2.040001
            ...   
851259    0.110001
851260    0.029999
851261    0.110000
851262   -0.720001
851263    0.570000
Length: 851264, dtype: float64

In [44]:
latent_dim = 32

autoencoder = AutoEncoder(data_shape=(cooperations_cnt,), latent_shape=(latent_dim,))
predictor = Network.dense(input_dim=autoencoder.latent_shape, output_dim=(1,))
autoencoder, predictor

(Sequential(
   (0): Reshape((501,))
   (1): Linear(in_features=501, out_features=344, bias=True)
   (2): ReLU()
   (3): Linear(in_features=344, out_features=188, bias=True)
   (4): ReLU()
   (5): Linear(in_features=188, out_features=32, bias=True)
   (6): Reshape((32,))
   (7): Reshape((32,))
   (8): Linear(in_features=32, out_features=188, bias=True)
   (9): ReLU()
   (10): Linear(in_features=188, out_features=344, bias=True)
   (11): ReLU()
   (12): Linear(in_features=344, out_features=501, bias=True)
   (13): Reshape((501,))
 ),
 Sequential(
   (0): Reshape((32,))
   (1): Linear(in_features=32, out_features=21, bias=True)
   (2): ReLU()
   (3): Linear(in_features=21, out_features=11, bias=True)
   (4): ReLU()
   (5): Linear(in_features=11, out_features=1, bias=True)
   (6): Reshape((1,))
 ))

In [10]:
autoencoder.adam().fit(
    X_train=X_train,
    Y_train=X_train,
    epochs=10_000,
    batch_size=256,
    X_val=X_val,
    Y_val=X_val,
    early_stop_count=1500,
    loss_criterion="MSELoss",
    verbose=True
).plot_loss("California housing autoencoder training")

Early stopping! Train-loss: 1440.316406, Val-loss: 1454.470215:  24%|██▍       | 2398/10000 [00:29<01:33, 81.59it/s]


In [61]:
predictor.adam().fit(
    X_train=autoencoder.encoder(X_train).output(),
    Y_train=Y_train,
    epochs=10_000,
    batch_size=8,
    X_val=autoencoder.encoder(X_val).output(),
    Y_val=Y_val,
    early_stop_count=1000,
    loss_criterion="MSELoss",
    verbose=True
).plot_loss("California housing predictor training")

Train-loss: 0.824268, Val-loss: 1.692824:   0%|          | 4/10000 [00:00<04:30, 36.94it/s]

Early stopping! Train-loss: 0.350845, Val-loss: 1.035278:  16%|█▋        | 1635/10000 [00:17<01:29, 93.20it/s] 


In [65]:
sample = X_val[np.random.randint(0, len(X_val))]

normal_explainer = (autoencoder.encoder + predictor).explainer("permutation", X_val)

reconstruction_explainer = autoencoder.decoder.explainer("exact", X_val)
predictor_explainer = predictor.explainer("exact", X_val)

In [66]:
predict = (autoencoder.encoder + predictor)(sample)

Y_hat = predict.output().numpy(force=True)
shap_values = predict.explain("deep", X_val)

/home/eivind/master-project/.venv/lib/python3.11/site-packages/shap/explainers/_deep/deep_pytorch.py:242: UserWarning:

unrecognized nn.Module: Reshape



In [70]:
shap_values.shap_values.sum()

0.0

In [135]:
autoencoder.encoder(X_val).output().mean(dim=0)

tensor([-1477.7847,  1017.0114,   680.9456], device='cuda:0',
       grad_fn=<MeanBackward1>)

In [127]:
autoencoder.encoder(sample).output()

tensor([-944.7205,  589.3659,  388.5021], device='cuda:0',
       grad_fn=<SqueezeBackward1>)

tensor([-1477.7847,  1017.0114,   680.9456], device='cuda:0',
       grad_fn=<MeanBackward1>)

In [106]:
shap_values[0].sum()

638.1976108152885

In [54]:
normal_explainer.explain(sample).list()[0].shap_values.tolist()

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]